In [35]:
import pandas as pd;
import numpy as np;
import sklearn as sk;
import xgboost as xgb;
import matplotlib as plt;
import seaborn as sns;
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score


In [6]:
#Cargamos el dataset desde el archivo csv
df = pd.read_csv(r'C:\Users\mrodr\OneDrive\Escritorio\UAL\TFG\prestamos campos definidos\loan.csv')

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\682258873.py:2: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\mrodr\OneDrive\Escritorio\UAL\TFG\prestamos campos definidos\loan.csv')


In [7]:
pd.set_option('display.max_columns', None)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0.0,Apr-2001,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,NaN,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0.0,Jun-1987,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt cons

In [8]:
#Filtramos por las siguientes variables:
columnas_deseadas = [
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 
    'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 
    'issue_d', 'loan_status', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 
    'earliest_cr_line', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 
    'initial_list_status', 'application_type', 'mort_acc', 'pub_rec_bankruptcies'
]

# Filtrar el DataFrame para quedarte solo con las columnas deseadas
df_filtrado = df[columnas_deseadas]


In [9]:
# Mostrar las primeras filas del DataFrame filtrado
df_filtrado.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,2500,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,109xx,NY,18.24,Apr-2001,9.0,1.0,4341,10.3,34.0,w,Individual,0.0,1.0
1,30000,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,713xx,LA,26.52,Jun-1987,13.0,1.0,12315,24.2,44.0,w,Individual,3.0,1.0
2,5000,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,490xx,MI,10.51,Apr-2011,8.0,0.0,4599,19.1,13.0,w,Individual,2.0,0.0
3,4000,36 months,18.94,146.51,D,D2,IT Supervisor,10+ years,MORTGAGE,92000.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,985xx,WA,16.74,Feb-2006,10.0,0.0,5468,78.1,13.0,w,Individual,3.0,0.0
4,30000,60 months,16.14,731.78,C,C4,Mechanic,10+ years,MORTGAGE,57250.0,Not Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,212xx,MD,26.35,Dec-2000,12.0,0.0,829,3.6,26.0,w,Individual,2.0,0.0


In [10]:
# Comprobar si hay valores nulos en el DataFrame
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Data columns (total 28 columns):
 #   Column                Dtype  
---  ------                -----  
 0   loan_amnt             int64  
 1   term                  object 
 2   int_rate              float64
 3   installment           float64
 4   grade                 object 
 5   sub_grade             object 
 6   emp_title             object 
 7   emp_length            object 
 8   home_ownership        object 
 9   annual_inc            float64
 10  verification_status   object 
 11  issue_d               object 
 12  loan_status           object 
 13  purpose               object 
 14  title                 object 
 15  zip_code              object 
 16  addr_state            object 
 17  dti                   float64
 18  earliest_cr_line      object 
 19  open_acc              float64
 20  pub_rec               float64
 21  revol_bal             int64  
 22  revol_util            float64
 23  total_a

In [11]:

# Identificamos las columnas de tipo object
columnas_object = df_filtrado.select_dtypes(include=['object']).columns

# Convertimos las columnas de tipo object a category
df_filtrado[columnas_object] = df_filtrado[columnas_object].astype('category')

# Verificamos los tipos de datos después de la conversión
print(df_filtrado.dtypes)

loan_amnt                  int64
term                    category
int_rate                 float64
installment              float64
grade                   category
sub_grade               category
emp_title               category
emp_length              category
home_ownership          category
annual_inc               float64
verification_status     category
issue_d                 category
loan_status             category
purpose                 category
title                   category
zip_code                category
addr_state              category
dti                      float64
earliest_cr_line        category
open_acc                 float64
pub_rec                  float64
revol_bal                  int64
revol_util               float64
total_acc                float64
initial_list_status     category
application_type        category
mort_acc                 float64
pub_rec_bankruptcies     float64
dtype: object


C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\3267504800.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[columnas_object] = df_filtrado[columnas_object].astype('category')


In [12]:
df_filtrado.home_ownership.value_counts()

home_ownership
MORTGAGE    1111450
RENT         894929
OWN          253057
ANY             996
OTHER           182
NONE             54
Name: count, dtype: int64

In [13]:
# Supongamos que tienes un DataFrame llamado data

# Mostrar el número de registros antes de eliminar filas con NaN
print(f"Número de registros antes de eliminar NaN: {len(df_filtrado)}")

# Eliminar filas con valores faltantes
data_limpia = df_filtrado.dropna()

# Mostrar el número de registros después de eliminar filas con NaN
print(f"Número de registros después de eliminar NaN: {len(data_limpia)}")

Número de registros antes de eliminar NaN: 2260668
Número de registros después de eliminar NaN: 2024454


In [14]:
data_limpia.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,2500,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,109xx,NY,18.24,Apr-2001,9.0,1.0,4341,10.3,34.0,w,Individual,0.0,1.0
1,30000,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,713xx,LA,26.52,Jun-1987,13.0,1.0,12315,24.2,44.0,w,Individual,3.0,1.0
2,5000,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,490xx,MI,10.51,Apr-2011,8.0,0.0,4599,19.1,13.0,w,Individual,2.0,0.0
3,4000,36 months,18.94,146.51,D,D2,IT Supervisor,10+ years,MORTGAGE,92000.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,985xx,WA,16.74,Feb-2006,10.0,0.0,5468,78.1,13.0,w,Individual,3.0,0.0
4,30000,60 months,16.14,731.78,C,C4,Mechanic,10+ years,MORTGAGE,57250.0,Not Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,212xx,MD,26.35,Dec-2000,12.0,0.0,829,3.6,26.0,w,Individual,2.0,0.0


In [15]:
# Seleccionar solo columnas numéricas
data_numerica = data_limpia.select_dtypes(include=['float64', 'int64'])

# Verificar si hay valores faltantes
print(data_numerica.isnull().sum())

# Eliminar filas con valores faltantes (si es necesario)
#data_numerica = data_numerica.dropna()
plt.figure(figsize=(12, 8))
sns.heatmap(data_numerica.corr(), annot=True, cmap='viridis')

loan_amnt               0
int_rate                0
installment             0
annual_inc              0
dti                     0
open_acc                0
pub_rec                 0
revol_bal               0
revol_util              0
total_acc               0
mort_acc                0
pub_rec_bankruptcies    0
dtype: int64


TypeError: 'module' object is not callable

In [16]:
data_limpia.head()


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,2500,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,109xx,NY,18.24,Apr-2001,9.0,1.0,4341,10.3,34.0,w,Individual,0.0,1.0
1,30000,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,713xx,LA,26.52,Jun-1987,13.0,1.0,12315,24.2,44.0,w,Individual,3.0,1.0
2,5000,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,490xx,MI,10.51,Apr-2011,8.0,0.0,4599,19.1,13.0,w,Individual,2.0,0.0
3,4000,36 months,18.94,146.51,D,D2,IT Supervisor,10+ years,MORTGAGE,92000.0,Source Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,985xx,WA,16.74,Feb-2006,10.0,0.0,5468,78.1,13.0,w,Individual,3.0,0.0
4,30000,60 months,16.14,731.78,C,C4,Mechanic,10+ years,MORTGAGE,57250.0,Not Verified,Dec-2018,Current,debt_consolidation,Debt consolidation,212xx,MD,26.35,Dec-2000,12.0,0.0,829,3.6,26.0,w,Individual,2.0,0.0


In [17]:
data_limpia['home_ownership'].value_counts()

home_ownership
MORTGAGE    1004710
RENT         805295
OWN          213628
ANY             737
NONE             43
OTHER            41
Name: count, dtype: int64

In [18]:
data_limpia.loc[(data_limpia.home_ownership == 'ANY') | (data_limpia.home_ownership == 'NONE'), 'home_ownership'] = 'OTHER'  
data_limpia.home_ownership.value_counts()

home_ownership
MORTGAGE    1004710
RENT         805295
OWN          213628
OTHER           821
ANY               0
NONE              0
Name: count, dtype: int64

In [19]:
print(data_limpia.shape)

(2024454, 28)


In [20]:
def pub_rec(number):
    if number == 0.0:
        return 0
    else:
        return 1
    
def mort_acc(number):
    if number == 0.0:
        return 0
    elif number >= 1.0:
        return 1
    else:
        return number
    
def pub_rec_bankruptcies(number):
    if number == 0.0:
        return 0
    elif number >= 1.0:
        return 1
    else:
        return number

In [21]:
data_limpia['pub_rec'] = data_limpia.pub_rec.apply(pub_rec)
data_limpia['mort_acc'] = data_limpia.mort_acc.apply(mort_acc)
data_limpia['pub_rec_bankruptcies'] = data_limpia.pub_rec_bankruptcies.apply(pub_rec_bankruptcies)

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\3639327639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia['pub_rec'] = data_limpia.pub_rec.apply(pub_rec)
C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\3639327639.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia['mort_acc'] = data_limpia.mort_acc.apply(mort_acc)
C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\3639327639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [22]:
print(data_limpia)

         loan_amnt        term  int_rate  installment grade sub_grade  \
0             2500   36 months     13.56        84.92     C        C1   
1            30000   60 months     18.94       777.23     D        D2   
2             5000   36 months     17.97       180.69     D        D1   
3             4000   36 months     18.94       146.51     D        D2   
4            30000   60 months     16.14       731.78     C        C4   
...            ...         ...       ...          ...   ...       ...   
2260662      30000   60 months     30.75       984.47     F        F5   
2260663      12000   60 months     14.08       279.72     C        C3   
2260664      12000   60 months     25.82       358.01     E        E4   
2260665      10000   36 months     11.99       332.10     B        B5   
2260667      16550   60 months     21.45       451.94     D        D5   

              emp_title emp_length home_ownership  annual_inc  \
0                  Chef  10+ years           RENT     5500

In [23]:
#Vemos cuantas variables únicas hay en la columna emp_title
data_limpia.emp_title.nunique()

481880

In [24]:
#Al haber cuantas variables hay, conviene mejor eliminar la columna
data_limpia.drop('emp_title', axis=1, inplace=True)

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\1124921106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia.drop('emp_title', axis=1, inplace=True)


In [25]:
data_limpia.drop('emp_length', axis=1, inplace=True)

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\2663674366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia.drop('emp_length', axis=1, inplace=True)


In [26]:
data_limpia.drop('title', axis=1, inplace=True)

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\2526413731.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia.drop('title', axis=1, inplace=True)


In [27]:
data_limpia.drop('grade', axis=1, inplace=True)

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\460814142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia.drop('grade', axis=1, inplace=True)


In [28]:
#cosas de la dirección
data_limpia.drop('addr_state', axis=1, inplace=True)

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\963959063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia.drop('addr_state', axis=1, inplace=True)


In [29]:
data_limpia.drop('issue_d', axis=1, inplace=True)

C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\1738250361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia.drop('issue_d', axis=1, inplace=True)


In [30]:
w_p = data_limpia.loan_status.value_counts()[0] / data_limpia.shape[0]
w_n = data_limpia.loan_status.value_counts()[1] / data_limpia.shape[0]

print(f"Weight of positive values {w_p}")
print(f"Weight of negative values {w_n}")

Weight of positive values 0.46071533361587863
Weight of negative values 0.4106292363274246


C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\3650594978.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w_p = data_limpia.loan_status.value_counts()[0] / data_limpia.shape[0]
C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\3650594978.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  w_n = data_limpia.loan_status.value_counts()[1] / data_limpia.shape[0]


In [31]:
data_limpia.loan_status.value_counts()

loan_status
Fully Paid                                             932697
Current                                                831300
Charged Off                                            229510
Late (31-120 days)                                      19528
In Grace Period                                          8186
Late (16-30 days)                                        3205
Default                                                    28
Does not meet the credit policy. Status:Fully Paid          0
Does not meet the credit policy. Status:Charged Off         0
Name: count, dtype: int64

In [32]:
# Definir las categorías de Good Loan y Bad Loan
good_loan_status = ["Current", "Fully Paid", "Does not meet the credit policy. Status:Fully Paid"]
bad_loan_status = ["Late (31-120 days)", "In Grace Period", "Charged Off", 
                   "Late (16-30 days)", "Default", 
                   "Does not meet the credit policy. Status:Charged Off"]

# Transformar 'loan_status' a binario
data_limpia['loan_status'] = data_limpia['loan_status'].apply(lambda x: 1 if x in good_loan_status else 0)

# Verificar la transformación
print(data_limpia['loan_status'].value_counts())
data_limpia.head()

loan_status
1    1763997
0     260457
Name: count, dtype: int64


C:\Users\mrodr\AppData\Local\Temp\ipykernel_3348\529177509.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpia['loan_status'] = data_limpia['loan_status'].apply(lambda x: 1 if x in good_loan_status else 0)


,loan_amnt,term,int_rate,installment,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,zip_code,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,2500,36 months,13.56,84.92,C1,RENT,55000.0,Not Verified,1,debt_consolidation,109xx,18.24,Apr-2001,9.0,1,4341,10.3,34.0,w,Individual,0,1
1,30000,60 months,18.94,777.23,D2,MORTGAGE,90000.0,Source Verified,1,debt_consolidation,713xx,26.52,Jun-1987,13.0,1,12315,24.2,44.0,w,Individual,1,1
2,5000,36 months,17.97,180.69,D1,MORTGAGE,59280.0,Source Verified,1,debt_consolidation,490xx,10.51,Apr-2011,8.0,0,4599,19.1,13.0,w,Individual,1,0
3,4000,36 months,18.94,146.51,D2,MORTGAGE,92000.0,Source Verified,1,debt_consolidation,985xx,16.74,Feb-2006,10.0,0,5468,78.1,13.0,w,Individual,1,0
4,30000,60 months,16.14,731.78,C4,MORTGAGE,57250.0,Not Verified,1,debt_consolidation,212xx,26.35,Dec-2000,12.0,0,829,3.6,26.0,w,Individual,1,0


In [33]:
w_p = data_limpia.loan_status.value_counts()[0] / data_limpia.shape[0]
w_n = data_limpia.loan_status.value_counts()[1] / data_limpia.shape[0]

print(f"Weight of positive values {w_p}")
print(f"Weight of negative values {w_n}")

Weight of positive values 0.12865543005669677
Weight of negative values 0.8713445699433032


In [34]:
data_limpia.head()

,loan_amnt,term,int_rate,installment,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,zip_code,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,2500,36 months,13.56,84.92,C1,RENT,55000.0,Not Verified,1,debt_consolidation,109xx,18.24,Apr-2001,9.0,1,4341,10.3,34.0,w,Individual,0,1
1,30000,60 months,18.94,777.23,D2,MORTGAGE,90000.0,Source Verified,1,debt_consolidation,713xx,26.52,Jun-1987,13.0,1,12315,24.2,44.0,w,Individual,1,1
2,5000,36 months,17.97,180.69,D1,MORTGAGE,59280.0,Source Verified,1,debt_consolidation,490xx,10.51,Apr-2011,8.0,0,4599,19.1,13.0,w,Individual,1,0
3,4000,36 months,18.94,146.51,D2,MORTGAGE,92000.0,Source Verified,1,debt_consolidation,985xx,16.74,Feb-2006,10.0,0,5468,78.1,13.0,w,Individual,1,0
4,30000,60 months,16.14,731.78,C4,MORTGAGE,57250.0,Not Verified,1,debt_consolidation,212xx,26.35,Dec-2000,12.0,0,829,3.6,26.0,w,Individual,1,0


In [36]:
# 1. Preparar los datos
X = data_limpia.drop(columns=['loan_status'])  # Variables independientes
y = data_limpia['loan_status']  # Variable objetivo

In [37]:
# 2. Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
# Crear y entrenar el modelo XGBoost
model = XGBClassifier(
    random_state=42,  # Semilla para reproducibilidad
    n_estimators=100,  # Número de árboles
    learning_rate=0.1,  # Tasa de aprendizaje
    max_depth=5,  # Profundidad máxima de los árboles
    subsample=0.8,  # Submuestra del conjunto de datos para cada árbol
    tree_method='hist',  # Método eficiente para datasets grandes
    colsample_bytree=0.8,  # Submuestra de características para cada árbol
    eval_metric='logloss',  # Métrica de evaluación
    enable_categorical=True,  # Habilitar el manejo de variables categóricas
    n_jobs=-1  # Usar todos los núcleos de la CPU
)

In [39]:
# Entrenar el modelo
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=None, random_state=42, ...)

In [40]:
# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para la clase positiva


In [41]:
# Evaluar el modelo
print("Precisión del modelo (XGBoost):", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nAUC-ROC:", roc_auc_score(y_test, y_pred_proba))

Precisión del modelo (XGBoost): 0.8713431516136442

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.50      0.01      0.01     52096
           1       0.87      1.00      0.93    352795

    accuracy                           0.87    404891
   macro avg       0.69      0.50      0.47    404891
weighted avg       0.82      0.87      0.81    404891


Matriz de confusión:
[[   289  51807]
 [   285 352510]]

AUC-ROC: 0.726004247309168
